Install Packages

In [29]:
import sys
import pandas as pd
import random
random.seed(10)
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [12]:
sys.path.append("D:\Fake_News_Detection")
print(sys.path)

['C:\\Python3\\python38.zip', 'C:\\Python3\\DLLs', 'C:\\Python3\\lib', 'C:\\Python3', 'd:\\Fake_News_Detection\\venv', '', 'd:\\Fake_News_Detection\\venv\\lib\\site-packages', 'd:\\Fake_News_Detection\\venv\\lib\\site-packages\\win32', 'd:\\Fake_News_Detection\\venv\\lib\\site-packages\\win32\\lib', 'd:\\Fake_News_Detection\\venv\\lib\\site-packages\\Pythonwin', 'D:\\Fake_News_Detection', 'D:\\Fake_News_Detection']


Download the dataset from Kaggle and load 

In [13]:
fake_csv = pd.read_csv(".\dataset\Fake.csv")
true_csv = pd.read_csv(".\dataset\True.csv")

In [14]:
fake_csv.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [15]:
true_csv.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Merge title and text, add label and drop title, text, subject, date column

In [16]:
fake_csv['label'] = 1
fake_csv['title+text'] = fake_csv['title'] + ' ' + fake_csv['text']
fake_csv = fake_csv.drop(columns=['title', 'text', 'subject', 'date'], axis=0)
fake_csv.shape

(23481, 2)

In [17]:
true_csv['label'] = 0
true_csv['title+text'] = true_csv['title'] + ' ' + true_csv['text']
true_csv = true_csv.drop(columns=['title', 'text', 'subject', 'date'], axis=0)
true_csv.shape

(21417, 2)

Sampling equal numbers from face_csv and true_csv

In [18]:
idx = [random.randrange(true_csv.shape[0]) for _ in range(1000)]
fake_csv_resampled = fake_csv.iloc[idx]
true_csv_resampled = true_csv.iloc[idx]

Remove stop words and Lemmetization using spaCy

In [19]:
# nlp = English() # Load English tokenizer, tagger, parser, NER and word vectors
nlp = en_core_web_sm.load()
fake_csv_resampled['title+text'].apply(nlp)
true_csv_resampled['title+text'].apply(nlp)
fc = fake_csv_resampled.copy()
tc = true_csv_resampled.copy()


In [20]:
# fc['token_list'] = fc['title+text'].str.split()
# tc['token_list'] = tc['title+text'].str.split()
# fc['token_list']

In [21]:
# fc['vocab'] = fc['token_list'].apply(lambda words: [nlp.vocab[word] for word in words])
# tc['vocab'] = tc['token_list'].apply(lambda words: [nlp.vocab[word] for word in words])
# tc['vocab']

In [22]:
fc['filtered_words'] = fc['title+text'].apply(lambda vocabs: [vocab.lemma_ for vocab in nlp(vocabs) if not vocab.is_stop])
tc['filtered_words'] = tc['title+text'].apply(lambda vocabs: [vocab.lemma_ for vocab in nlp(vocabs) if not vocab.is_stop])
fc['filtered_words']

18723    [TRUMP, DELIVERS, BARN, BURNER, speech, :, ", ...
1067     [ , come, Cory, Booker, Pals, ', Field, trip, ...
14053    [obama, BRITAIN, Legacy, :, save, World, Econo...
15812    [[, video, ], watch, marco, rubio, Handles, re...
18942    [disturb, :, Tomi, Lahren, Panders, ", View, "...
                               ...                        
37       [ , Donald, Trump, Eating, habit, dramatically...
12230    [JILL, STEIN, Claims, Recount, possible, Forei...
8251     [ , state, let, gun, right, People, think, thi...
19081    [MAJOR, COSMETIC, Company, announce, plan, rel...
19067    [illegal, alien, protester, yell, Police, shut...
Name: filtered_words, Length: 1000, dtype: object

Removing Non-alphanumeric characters

In [23]:
fc['cleaned_lemmas'] = fc['filtered_words'].apply(lambda word_list:[word for word in word_list if word.isalpha()])
tc['cleaned_lemmas'] = tc['filtered_words'].apply(lambda word_list:[word for word in word_list if word.isalpha()])
tc['cleaned_lemmas']

18723    [migrant, smuggling, crackdown, trigger, clash...
1067     [Schumer, cite, untruth, Trump, Republicans, t...
14053    [urge, Australia, protect, refugee, Papua, New...
15812    [Russia, tell, unacceptable, politicize, chemi...
18942    [thai, PM, Yingluck, get, year, jail, term, ne...
                               ...                        
37       [factbox, big, ticket, item, center, Congress,...
12230    [Russia, Egypt, talk, resumption, date, russia...
8251     [invoke, religion, disciplined, Trump, speak, ...
19081    [alliance, say, russian, jet, strike, fighter,...
19067    [turnout, high, iraqi, Kurds, defy, threat, ho...
Name: cleaned_lemmas, Length: 1000, dtype: object

In [36]:
df = pd.concat([fc,tc], axis=0)
df['text'] = df['cleaned_lemmas'].str.join(' ')
df = df.drop(columns=['title+text', 'filtered_words', 'cleaned_lemmas'])
df.head()

,label,text
18723,1,TRUMP DELIVERS BARN BURNER speech media outlet...
1067,1,come Cory Booker Pals Field trip find Super Se...
14053,1,obama BRITAIN Legacy save World Economy Deal I...
15812,1,video watch marco rubio Handles reporter Attem...
18942,1,disturb Tomi Lahren Panders View Hags say Pro ...


Feature Selection and split the data into test-train sets

In [38]:
X = df['text']
y = df['label']


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

Pipeine with TfidVectorizer and selection of Model

In [40]:
classifier = Pipeline([
    ("vectorizer_tfidf", TfidfVectorizer()),
    ("random_forest", RandomForestClassifier(n_jobs=-1, random_state=42))
])
classifier.fit(X_train,y_train)


Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('random_forest',
                 RandomForestClassifier(n_jobs=-1, random_state=42))])

In [42]:
accuracy = classifier.score(X_test, y_test) * 100
accuracy

98.5

In [44]:
pred = classifier.predict(X_test)
classification_report(y_test, pred)

'              precision    recall  f1-score   support\n\n           0       0.98      1.00      0.99       201\n           1       0.99      0.97      0.98       199\n\n    accuracy                           0.98       400\n   macro avg       0.99      0.98      0.98       400\nweighted avg       0.99      0.98      0.98       400\n'